# Query -> Category w/ No Category Found

<small>
(from <a href="http://maven.com/softwaredoug/cheat-at-search">Cheat at Search with LLMs</a> training course by Doug Turnbull.)
</small>

Previously we [categorized queries](https://colab.research.google.com/drive/1cPG0l-lCOPKDpCAJkhyNCwTjgTmCtzRz). In this notebook, we add a refinement to improve precision, but sacrificing recall. We let the LLM say it doesn't know by returning "No Category Found".

This might let us get some of the benefit with less of the downside.


## Boilerplate

Install deps, mount GDrive, prompt for your OpenAI Key (placed in your GDrive), and import needed cheat at search helpers.

We cover this extensively in the [synonyms notebook](https://colab.research.google.com/drive/1aUCvcBa1YdmsbIgYc74jlknl9_iRotp1) walkthrough

In [ ]:
!pip install git+https://github.com/softwaredoug/cheat-at-search.git
from cheat_at_search.data_dir import mount
mount(use_gdrive=True)
from cheat_at_search.search import run_strategy, graded_bm25, ndcgs, ndcg_delta, vs_ideal
from cheat_at_search.wands_data import products

products

  Cloning https://github.com/softwaredoug/cheat-at-search.git to /tmp/pip-req-build-o8a1aody
  Running command git clone --filter=blob:none --quiet https://github.com/softwaredoug/cheat-at-search.git /tmp/pip-req-build-o8a1aody
  Resolved https://github.com/softwaredoug/cheat-at-search.git to commit 38a087b480422fb5f29fea8b25fbfb25f3492da3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 4.4 MB/s eta 0:00:00
  Created wheel for cheat_at_search: filename=cheat_at_search-0.1.0-py3-none-any.whl size=40951 sha256=8418acb416c9862f0b3fa4692cd9d08d55a5bb825fbf6e9026c2c056005c31cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-t7jmhz8o/wheels/ea/96/d3/e69453e5f86e6f891864e5b5baf2b11ffda2d0981d9f8a138e
Successfully built cheat_at_search
Mounted at /content/drive


,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,category,sub_category,cat_subcat
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0,"[overallwidth-sidetoside:64.7, dsprimaryproduc...",Furniture,Bedroom Furniture,Furniture / Bedroom Furniture
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0,"[capacityquarts:7, producttype : slow cooker, ...",Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0,"[features : keep warm setting, capacityquarts:...",Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0,"[overallwidth-sidetoside:3.5, warrantylength :...",Browse By Brand,All-Clad,Browse By Brand / All-Clad
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0,"[compatibledoorthickness:1.375 '' , countryofo...",Home Improvement,Doors & Door Hardware,Home Improvement / Doors & Door Hardware
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0,"[producttype : shower panel, spraypattern : ra...",Home Improvement,Bathroom Remodel & Bathroom Fixtures,Home Improvement / Bathroom Remodel & Bathro...
42990,42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0,"[basematerialdetails : steel, : gray wood, of...",Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture
42991,42991,maloney 3 piece pub table set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,this pub table set includes 1 counter height t...,additionaltoolsrequirednotincluded : power dri...,49.0,4.0,41.0,[additionaltoolsrequirednotincluded : power dr...,Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture
42992,42992,fletcher 27.5 '' wide polyester armchair,Teen Lounge Furniture|Accent Chairs,Furniture / Living Room Furniture / Chairs & S...,"bring iconic , modern style to your space in a...",legmaterialdetails : rubberwood|backheight-sea...,1746.0,4.5,1226.0,"[legmaterialdetails : rubberwood, backheight-s...",Furniture,Living Room Furniture,Furniture / Living Room Furniture


## Query -> Category classification (allow no category found)

Iterating on the previous notebook, this code is identical. However we have added No Category Found / No SubCategory Found. Note the comments of what's ADDED/CHANGED from before.

**What's the goal?** -- search is a constant precision / recall tradeoff. Here, we know there is big downside to misclassification. What if we mitigate that downside by giving the LLM the ability to NOT classify to a category.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal
from cheat_at_search.enrich import AutoEnricher


CategoriesWithUnknown = Literal['Furniture',
                     'Home Improvement',
                     'Décor & Pillows',
                     'Outdoor',
                     'Storage & Organization',
                     'Lighting',
                     'Rugs',
                     'Bed & Bath',
                     'Kitchen & Tabletop',
                     'Baby & Kids',
                     'School Furniture and Supplies',
                     'Appliances',
                     'Holiday Décor',
                     'Commercial Business Furniture',
                     'Pet',
                     'Contractor',
                     'Sale',
                     'Foodservice ',
                     'Reception Area',
                     'Clips',
                     'No Category Fits']   #< ADDDED!

SubCategoriesWithUnknown = Literal['Bedroom Furniture',
 'Small Kitchen Appliances',
 'All-Clad',
 'Doors & Door Hardware',
 'Bathroom Remodel & Bathroom Fixtures',
 'Home Accessories',
 'Living Room Furniture',
 'Outdoor Décor',
 'Flooring, Walls & Ceiling',
 'Garage & Outdoor Storage & Organization',
 'Cookware & Bakeware',
 'Bedding',
 'Kitchen Utensils & Tools',
 'Shower Curtains & Accessories',
 'Wall Shelving & Organization',
 'Clocks',
 'Bedding Essentials',
 'Kitchen & Dining Furniture',
 'Office Furniture',
 'Tableware & Drinkware',
 'Nursery Bedding',
 'Cat',
 'Outdoor Shades',
 'Outdoor & Patio Furniture',
 'Ceiling Lights',
 'Area Rugs',
 'Outdoor Lighting',
 'Window Treatments',
 'Garden',
 'Closet Storage & Organization',
 'Wall Décor',
 'Mirrors',
 'Shoe Storage',
 'Toddler & Kids Playroom',
 'Game Tables & Game Room Furniture',
 'Decorative Pillows & Blankets',
 'School Furniture',
 'Wall Lights',
 'Bathroom Storage & Organization',
 'Commercial Office Furniture',
 'Flowers & Plants',
 'Mattresses & Foundations',
 'Area Rugs',
 'Cleaning & Laundry Organization',
 'Kitchen Organization',
 'Candles & Holders',
 'Christmas',
 'Toddler & Kids Bedroom Furniture',
 'Front Door Décor & Curb Appeal',
 'Storage Furniture',
 'School Spaces',
 'Hardware',
 'Light Bulbs & Hardware',
 'Ceiling Fans',
 'Doormats',
 'Entry & Hallway',
 'Storage Containers & Drawers',
 'Holiday Lighting',
 'Kitchen Mats',
 'Facilities & Maintenance',
 'Table & Floor Lamps',
 'Bird',
 'Kitchen Appliances',
 'Building Equipment',
 'Art',
 'Picture Frames & Albums',
 'Outdoor Heating',
 'Outdoor Recreation',
 'Bathroom Accessories & Organization',
 'School Boards & Technology',
 'Closeout',
 'Reception Seating',
 'Foodservice Tables',
 'Kitchen Remodel & Kitchen Fixtures',
 'Hot Tubs & Saunas',
 'Teen Bedroom Furniture',
 'Outdoor Fencing & Flooring',
 'Chairs',
 'Bath Rugs & Towels',
 'Fish',
 'Dog',
 'Chicken',
 'Boards & Tech Accessories',
 'Commercial Contractor',
 'Clamps',
 'Jewelry Organization',
 'Entry & Mudroom Furniture',
 'Outdoor Cooking & Tableware',
 'Seasonal Décor',
 'Nursery Furniture',
 'Storage & Organization Sale',
 'Washers & Dryers',
 'Baby & Kids Décor & Lighting',
 'Outdoor Remodel',
 'Plumbing',
 'Birch Lane™',
 'Office Organization',
 'Kitchen & Dining Sale',
 'Holiday Lighting',
 'Baby & Kids Storage',
 'Shop All Characters',
 'Commercial Kitchen',
 'Guest Room Amenities',
 'Charlton Home',
 'Wade Logan®',
 'Heating, Cooling & Air Quality',
 'Thanksgiving',
 'Fourth of July',
 'Vacuums & Deep Cleaners',
 'Stair Tread Rugs',
 'Small Spaces',
 'Toddler & Kids Bedding & Bath',
 'Classroom Décor',
 'Early Education Play Area',
 'Zoomie Kids',
 'Fryers',
 'August Grove',
 'Dorm Décor & Back to School Essentials',
 'Symple Stuff',
 'Wayfair Basics®',
 'The Holiday Aisle',
 'Chair Pads & Cushions',
 'The Monogram Shop',
 'Wedding',
 'Wedding',
 'Reception Desks & Tables',
 'Rug Pads',
 'Latitude Run',
 'Accommodations Furniture',
 'Easter',
 'Furniture Sale',
 'Shop All Characters',
 'Novelty Lights',
 "Valentine's Day",
 'Outdoor Sale',
 'Classroom & Training Furniture',
 'Rebrilliant',
 'Rug Pads',
 'Commercial Kitchen Storage',
 'Teen Bedding',
 'Tommy Bahama Home',
 'Appliances Sale',
 'Massage Products',
 'No SubCategory Fits']   #< ADDDED!


class Query(BaseModel):
    """
    Base model for search queries, containing common query attributes.
    """
    keywords: str = Field(
        ...,
        description="The original search query keywords sent in as input"
    )


class QueryCategoryWithUnknown(Query):
    """
    Structured representation of a search query for furniture e-commerce.
    Inherits keywords from the base Query model and adds category and sub-category.
    """
    category: CategoriesWithUnknown = Field(
        description="Category of the product, if identified. Use 'No Category Fits' if ambiguous or no category in list fits"
    ) # CHANGED - description allows classification to unknown
    sub_category: SubCategoriesWithUnknown = Field(
        description="Sub-category of the product, if identified. Use 'No SubCategory Fits' if ambiguous or no sub-category in list fits"
    ) # CHANGED - description allows classification to unknown

    @property
    def classification(self) -> str:
        return f"{self.category} / {self.sub_category}"





### Query classification code

Our code here is identical to

In [ ]:
enricher = AutoEnricher(
     model="openai/gpt-4o",
     system_prompt="You are a helpful furniture shopping agent that helps users construct search queries.",
     response_model=QueryCategoryWithUnknown
)

def get_prompt_with_unknown(query):
    prompt = f"""
        As a helpful agent, you'll recieve requests from users looking for furniture products.

        Your task is to search with a structured query against a furniture product catalog.

        Here is the users request:

        {query}

        Return Category / Subcategory:

        * Category - the category (as listed in schema) for the query.
        * SubCategory - the subcategory (as listed in the schema) for the query

        Use "No Category Fits" / "No SubCategory Fits" if ambiguous, unclear, or many might fit.
    """
    return prompt


def categorized_with_unknown(query):
    prompt = get_prompt_with_unknown(query)
    return enricher.enrich(prompt)

categorized_with_unknown("banana popsicle")

QueryCategoryWithUnknown(keywords='banana popsicle', category='No Category Fits', sub_category='No SubCategory Fits')

### Define ground truth

As in the previous notebook, we re-define our ground truth for query classification. We define `prec_cat` to measure precision.

In [ ]:
from cheat_at_search.wands_data import labeled_query_products, queries

def get_top_category(column, no_fit_label, cutoff=0.8):
    # Get relevant products per query
    top_products = labeled_query_products[labeled_query_products['grade'] == 2]

    # Aggregate top categories
    categories_per_query_ideal = top_products.groupby('query')[column].value_counts().reset_index()

    # Get as percentage of all categories for this query
    top_cat_proportion = categories_per_query_ideal.groupby(['query', column]).sum() / categories_per_query_ideal.groupby('query').sum()
    top_cat_proportion = top_cat_proportion.drop(columns=column).reset_index()

    # Only look at cases where the category is > 0.8
    top_cat_proportion = top_cat_proportion[top_cat_proportion['count'] > cutoff]
    top_cat_proportion[column].fillna(no_fit_label, inplace=True)
    ground_truth_cat = top_cat_proportion
    # Give No Category Fits to all others without dominant category
    ground_truth_cat = ground_truth_cat.merge(queries, how='right', on='query')[['query', column, 'count']]
    ground_truth_cat[column].fillna(no_fit_label, inplace=True)
    return ground_truth_cat

ground_truth_cat = get_top_category('category', 'No Category Fits')
ground_truth_cat

/tmp/ipython-input-3280341426.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  top_cat_proportion[column].fillna(no_fit_label, inplace=True)
/tmp/ipython-input-3280341426.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

,query,category,count
0,salon chair,No Category Fits,NaN
1,smart coffee table,Furniture,1.000000
2,dinosaur,No Category Fits,NaN
3,turquoise pillows,Décor & Pillows,0.963636
4,chair and a half recliner,Furniture,0.956522
...,...,...,...
475,rustic twig,Décor & Pillows,1.000000
476,nespresso vertuo next premium by breville with...,No Category Fits,NaN
477,pedistole sink,No Category Fits,NaN
478,54 in bench cushion,No Category Fits,NaN


### Modified precision -- measure coverage / recall

We'll measure both precision -- accuracy when prediction made -- and coverage -- how many items are given predictions.

In [ ]:
def get_pred(cat, column):
    if column == 'category':
        return cat.category
    elif column == 'sub_category':
        return cat.sub_category
    else:
        raise ValueError(f"Unknown column {column}")


def prec_cat(ground_truth, column, no_fit_label, categorized, N=500):
    hits = []
    misses = []
    for _, row in ground_truth.sample(frac=1).iterrows():
        query = row['query']
        expected_category = row[column]

        cat = categorized(query)
        pred = get_pred(cat, column)
        if pred == no_fit_label:
            print(f"Skipping {query}")
            continue
        if pred == expected_category.strip():
            hits.append((expected_category, cat))
        else:
            print("***")
            print(f"{query} -- predicted:{cat.category} != expected:{expected_category.strip()}")
            misses.append((expected_category, cat))
            num_so_far = len(hits) + len(misses)
            print(f"prec (N={num_so_far}) -- {len(hits) / (len(hits) + len(misses))}")
            print(f"coverage {num_so_far / len(ground_truth)}")

        if len(hits) + len(misses) > N:
            break
    return len(hits) / (len(hits) + len(misses)), num_so_far / len(ground_truth)

prec, coverage = prec_cat(ground_truth_cat, 'category', 'No Category Fits', categorized_with_unknown, N=500)
prec, coverage

Skipping kisner
***
cake plates with tops -- predicted:Kitchen & Tabletop != expected:No Category Fits
prec (N=1) -- 0.0
coverage 0.0020833333333333333
***
desk and chair set -- predicted:Furniture != expected:No Category Fits
prec (N=2) -- 0.0
coverage 0.004166666666666667
***
tall storage cabinet -- predicted:Storage & Organization != expected:No Category Fits
prec (N=3) -- 0.0
coverage 0.00625
Skipping merlyn 6
Skipping industrial
***
board game storage cabinet -- predicted:Storage & Organization != expected:Furniture
prec (N=7) -- 0.42857142857142855
coverage 0.014583333333333334
***
gray leather cocktail table -- predicted:Furniture != expected:No Category Fits
prec (N=8) -- 0.375
coverage 0.016666666666666666
Skipping promo codes or discounts
***
54 in bench cushion -- predicted:Furniture != expected:No Category Fits
prec (N=10) -- 0.4
coverage 0.020833333333333332
***
acrylic clear chair -- predicted:Furniture != expected:No Category Fits
prec (N=11) -- 0.36363636363636365
cover

(0.5658536585365853, 0.8541666666666666)

## Run Category search strategy with classifier

Our search strategy here is identical, **however note** when the LLM does not predict a category / sub category we do not provide a boost.

In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer
from cheat_at_search.strategy.strategy import SearchStrategy
import numpy as np


class CategorySearch(SearchStrategy):
    def __init__(self, products, query_to_cat,
                 name_boost=9.3,
                 description_boost=4.1,
                 category_boost=10,
                 sub_category_boost=5):
        super().__init__(products)
        self.index = products
        self.index['product_name_snowball'] = SearchArray.index(
            products['product_name'], snowball_tokenizer)
        self.index['product_description_snowball'] = SearchArray.index(
            products['product_description'], snowball_tokenizer)

        cat_split = products['category hierarchy'].fillna('').str.split("/")

        products['category'] = cat_split.apply(
            lambda x: x[0].strip() if len(x) > 0 else ""
        )
        products['subcategory'] = cat_split.apply(
            lambda x: x[1].strip() if len(x) > 1 else ""
        )
        self.index['category_snowball'] = SearchArray.index(
            products['category'], snowball_tokenizer
        )
        self.index['subcategory_snowball'] = SearchArray.index(
            products['subcategory'], snowball_tokenizer
        )

        self.query_to_cat = query_to_cat
        self.name_boost = name_boost
        self.description_boost = description_boost
        self.category_boost = category_boost
        self.sub_category_boost = sub_category_boost

    def search(self, query, k=10):
        """Dumb baseline lexical search, but add a constant boost when
           the desired category or subcategory"""
        bm25_scores = np.zeros(len(self.index))
        structured = self.query_to_cat(query)
        tokenized = snowball_tokenizer(query)

        # ****
        # Baseline BM25 search from before
        for token in tokenized:
            bm25_scores += self.index['product_name_snowball'].array.score(token) * self.name_boost
            bm25_scores += self.index['product_description_snowball'].array.score(
                token) * self.description_boost

        # ****
        # If there's a subcategory, boost that by a constant amount
        if structured.sub_category and structured.sub_category != "No SubCategory Fits":     # CHANGED -- CHECK IF WE MADE A PREDICTION
            tokenized_subcategory = snowball_tokenizer(structured.sub_category)
            subcategory_match = np.ones(len(self.index))
            if tokenized_subcategory:
                subcategory_match = self.index['subcategory_snowball'].array.score(tokenized_subcategory) > 0
            bm25_scores[subcategory_match] += self.sub_category_boost

        # ****
        # If there's a category, boost that by a constant amount
        if structured.category and structured.category != "No Category Fits":     #  CHANGED -- CHECK IF WE MADE A PREDICTION
            tokenized_category = snowball_tokenizer(structured.category)
            category_match = np.ones(len(self.index))
            if tokenized_category:
                category_match = self.index['category_snowball'].array.score(tokenized_category) > 0
            bm25_scores[category_match] += self.category_boost

        top_k = np.argsort(-bm25_scores)[:k]
        scores = bm25_scores[top_k]

        return top_k, scores


In [ ]:
categorized_search = CategorySearch(products, categorized_with_unknown)
graded_categorized = run_strategy(categorized_search)
graded_categorized

2025-10-13 17:19:43,063 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-10-13 17:19:43,089 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-10-13 17:19:43,093 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

2025-10-13 17:20:02,510 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-10-13 17:20:02,533 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete
Searching: 100%|██████████| 480/480 [00:13<00:00, 36.28it/s]


,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,...,query_id,rank,query_class,id,label,grade,discounted_gain,idcg,dcg,ndcg
0,7465,hair salon chair,Massage Chairs|Recliners,Furniture / Living Room Furniture / Chairs & S...,offers a wide selection of professional salon ...,fauxleathertype : pu|legheight-toptobottom:18|...,69.0,4.5,53.0,"[fauxleathertype : pu, legheight-toptobottom:1...",...,0,1,Massage Chairs,80.0,Exact,2.0,3.00,8.786905,8.10119,0.921962
1,25431,barberpub salon massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,salon chairs are a wonderful avenue for hairst...,supplierintendedandapproveduse : non residenti...,4.0,5.0,4.0,[supplierintendedandapproveduse : non resident...,...,0,2,Massage Chairs,29.0,Exact,2.0,1.50,8.786905,8.10119,0.921962
2,7468,mercer41 hair salon chair hydraulic styling ch...,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,mercer41 beauty offers a wide selection profes...,seatfillmaterial : foam|waterrepellant : no re...,1.0,5.0,1.0,"[seatfillmaterial : foam, waterrepellant : no ...",...,0,3,Massage Chairs,104.0,Exact,2.0,1.00,8.786905,8.10119,0.921962
3,39461,professional salon reclining massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,new and in a good condition . first-rate metal...,overalldepth-fronttoback:39.4|warrantylength:1...,NaN,NaN,NaN,"[overalldepth-fronttoback:39.4, warrantylength...",...,0,4,Massage Chairs,114.0,Exact,2.0,0.75,8.786905,8.10119,0.921962
4,9234,beauty salon task chair,NaN,Furniture / Office Furniture / Office Chairs,"applicable scene : office , home life , beauty...",overallheight-toptobottom:37|backcolor : brown...,NaN,NaN,NaN,"[overallheight-toptobottom:37, backcolor : bro...",...,0,5,Massage Chairs,32.0,Partial,1.0,0.20,8.786905,8.10119,0.921962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,22194,wine glass rack,Kitchen Sink Storage,Kitchen & Tabletop / Kitchen Organization / Co...,drip-dry up to eight wineglasses with this cle...,glasscapacity:8|countryoforigin : united state...,5.0,4.5,3.0,"[glasscapacity:8, countryoforigin : united sta...",...,487,6,NaN,NaN,NaN,0.0,0.00,8.786905,0.00000,0.000000
4796,40243,madisen hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,complement your farmhouse kitchen decor with t...,producttype : wine glass rack|overallwidth-sid...,29.0,5.0,20.0,"[producttype : wine glass rack, overallwidth-s...",...,487,7,NaN,NaN,NaN,0.0,0.00,8.786905,0.00000,0.000000
4797,40244,kena hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,spruce up your farmhouse kitchen decor with th...,warrantylength:1 year|producttype : wine glass...,23.0,5.0,18.0,"[warrantylength:1 year, producttype : wine gla...",...,487,8,NaN,NaN,NaN,0.0,0.00,8.786905,0.00000,0.000000
4798,39976,wall mounted wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,"the latest addition to this collection , this ...",overallheight-toptobottom:4|design : wall moun...,34.0,4.5,18.0,"[overallheight-toptobottom:4, design : wall mo...",...,487,9,NaN,NaN,NaN,0.0,0.00,8.786905,0.00000,0.000000


### Analyze results

We note:
1. NDCG down a bit from earlier
2. Though fewer queries have been harmed

In [ ]:
ndcgs(graded_bm25).mean(), ndcgs(graded_categorized).mean()

(np.float64(0.5411098691836396), np.float64(0.55962898357645))

In [ ]:
deltas = ndcg_delta(graded_categorized, graded_bm25)
deltas

,ndcg
query,
bathroom freestanding cabinet,0.692589
non slip shower floor tile,0.477081
outdoor lounge chair,0.456442
modern outdoor furniture,0.365533
twin bed frame,0.359391
...,...
whimsical solar lights,-0.142257
outdoor welcome rug,-0.158244
sheffield home bath set,-0.219708


In [ ]:
sig_improved = len(deltas[deltas > 0.1])
print(f"Num Significatly Improved: {sig_improved}")
deltas[deltas > 0.1]

Num Significatly Improved: 38


,ndcg
query,
bathroom freestanding cabinet,0.692589
non slip shower floor tile,0.477081
outdoor lounge chair,0.456442
modern outdoor furniture,0.365533
twin bed frame,0.359391
desk for kids,0.344262
wood rack wide,0.335320
outdoor light fixtures,0.311069
turquoise chair,0.298334


In [ ]:
sig_harmed = len(deltas[deltas < -0.1])
print(f"Num Significatly Harmed: {sig_harmed}")
print(f"Prop improved/harmed: {sig_improved / (sig_harmed + sig_improved)} | {sig_harmed / (sig_harmed + sig_improved)}")
deltas[deltas < -0.1]

Num Significatly Harmed: 8
Prop improved/harmed: 0.8260869565217391 | 0.17391304347826086


,ndcg
query,
zodiac pillow,-0.113806
papasan chair frame only,-0.123290
tall storage cabinet,-0.127715
whimsical solar lights,-0.142257
outdoor welcome rug,-0.158244
sheffield home bath set,-0.219708
outdoor lounge cushions,-0.471391
chair pillow cushion,-0.487874


### Analyze a query

Let's look at a negative query to see how its harmed.

In [ ]:
QUERY = "chair pillow cushion"
graded_bm25[graded_bm25['query'] == QUERY][['product_name', 'product_description', 'category hierarchy', 'grade']]

,product_name,product_description,category hierarchy,grade
4660,replacement pillows outdoor lounge chair cushion,this replacement pillows outdoor lounge chair ...,NaN,2.0
4661,indoor/outdoor dining chair cushion and pillow...,go bold and spicy with a fun geometric print o...,Outdoor / Outdoor Décor / Outdoor Pillows & Cu...,2.0
4662,abbottsmoor dining chair cushion,the dining chair cushion ( set of 4 ) is apt f...,NaN,2.0
4663,zipparoll indoor chair cushion,zips from round pillow to flat pillow . the zi...,Kitchen & Tabletop / Tableware & Drinkware / T...,2.0
4664,indoor chair cushion,brighten your indoor seating area with this se...,NaN,2.0
4665,chair pad cushion,are your dining chair ’ s feeling a little sti...,NaN,2.0
4666,chair indoor seat cushion,add a splash of vibrant color and radiant styl...,NaN,2.0
4667,chair outdoor seat cushion,add a splash of vibrant color and radiant styl...,NaN,2.0
4668,dining chair cushion,add a splash of personality and create a cozy ...,Kitchen & Tabletop / Tableware & Drinkware / T...,2.0
4669,tropical outdoor lounge chair cushion,enhance your outdoor space with the addition o...,NaN,2.0


In [ ]:
graded_categorized[graded_categorized['query'] == QUERY][['product_name', 'product_description', 'category hierarchy', 'grade']]

,product_name,product_description,category hierarchy,grade
4660,replacement pillows outdoor lounge chair cushion,this replacement pillows outdoor lounge chair ...,NaN,2.0
4661,indoor/outdoor dining chair cushion and pillow...,go bold and spicy with a fun geometric print o...,Outdoor / Outdoor Décor / Outdoor Pillows & Cu...,2.0
4662,peacock throw pillow,decorative pillow – designed by suren nersisya...,Décor & Pillows / Decorative Pillows & Blanket...,0.0
4663,two ocztopus throw pillow,decorative pillow – designed by suren nersisya...,Décor & Pillows / Decorative Pillows & Blanket...,0.0
4664,cancer zodiac throw pillow,decorative pillow – designed by emanuela carra...,Décor & Pillows / Decorative Pillows & Blanket...,0.0
4665,velvet ikat 3 '' lumbar pillow,bring the world to your sofa with this pillow ...,Décor & Pillows / Decorative Pillows & Blanket...,0.0
4666,finesse ii throw pillow,this decorative pillow is designed by pi creat...,Décor & Pillows / Decorative Pillows & Blanket...,0.0
4667,navy throw pillow,solid navy decorative accent throw pillows fro...,Décor & Pillows / Decorative Pillows & Blanket...,0.0
4668,marble petroleum ii throw pillow,decorative pillow – designed by pi creative ar...,Décor & Pillows / Decorative Pillows & Blanket...,0.0
4669,marble throw pillow,black and white marble collection decorative a...,Décor & Pillows / Decorative Pillows & Blanket...,0.0


In [ ]:
categorized_with_unknown(QUERY)

QueryCategoryWithUnknown(keywords='chair pillow cushion', category='Décor & Pillows', sub_category='Decorative Pillows & Blankets')

### Analyze Ground Truth for important queries

It's a bit surprising given higher precision in our classifier, NDCG would not improve.

Let's focus in on the relevant queries to see how they're impacted.

In [ ]:
impacted_queries = [
 'drum picture',
 'bathroom freestanding cabinet',
 'outdoor lounge chair',
 'wood rack wide',
 'outdoor light fixtures',
 'bathroom vanity knobs',
 'door jewelry organizer',
 'beds that have leds',
 'non slip shower floor tile',
 'turquoise chair',
 'modern outdoor furniture',
 'podium with locking cabinet',
 'closet storage with zipper',
 'barstool patio sets',
 'ayesha curry kitchen',
 'led 60',
 'wisdom stone river 3-3/4',
 'liberty hardware francisco',
 'french molding',
 'glass doors for bath',
 'accent leather chair',
 'dark gray dresser',
 'wainscoting ideas',
 'floating bed',
 'dining table vinyl cloth',
 'entrance table',
 'storage dresser',
 'almost heaven sauna',
 'toddler couch fold out',
 'outdoor welcome rug',
 'wooden chair outdoor',
 'emma headboard',
 'outdoor privacy wall',
 'driftwood mirror',
 'white abstract',
 'bedroom accessories',
 'bathroom lighting',
 'light and navy blue decorative pillow',
 'gnome fairy garden',
 'medium size chandelier',
 'above toilet cabinet',
 'odum velvet',
 'ruckus chair',
 'modern farmhouse lighting semi flush mount',
 'teal chair',
 'bedroom wall decor floral, multicolored with some teal (prints)',
 'big basket for dirty cloths',
 'milk cow chair',
 'small wardrobe grey',
 'glow in the dark silent wall clock',
 'medium clips',
 'desk for kids tjat ate 10 year old',
 'industrial pipe dining  table',
 'itchington butterfly',
 'midcentury tv unit',
 'gas detector',
 'fleur de lis living candle wall sconce bronze',
 'zodiac pillow',
 'papasan chair frame only',
 'bed side table']
prec, coverage = prec_cat(ground_truth_cat[ground_truth_cat['query'].isin(impacted_queries)],
                              'category', 'No Category Fits', categorized_with_unknown, N=500)
prec

Skipping milk cow chair
***
podium with locking cabinet -- predicted:Commercial Business Furniture != expected:School Furniture and Supplies
prec (N=2) -- 0.5
coverage 0.03333333333333333
Skipping french molding
Skipping liberty hardware francisco
Skipping ruckus chair
***
emma headboard -- predicted:Furniture != expected:Baby & Kids
prec (N=6) -- 0.6666666666666666
coverage 0.1
Skipping wisdom stone river 3-3/4
Skipping itchington butterfly
***
door jewelry organizer -- predicted:Storage & Organization != expected:Furniture
prec (N=11) -- 0.7272727272727273
coverage 0.18333333333333332
***
gnome fairy garden -- predicted:Décor & Pillows != expected:Outdoor
prec (N=19) -- 0.7894736842105263
coverage 0.31666666666666665
Skipping white abstract
Skipping odum velvet
***
outdoor welcome rug -- predicted:Outdoor != expected:Rugs
prec (N=22) -- 0.7727272727272727
coverage 0.36666666666666664
Skipping wainscoting ideas
Skipping gas detector
***
outdoor light fixtures -- predicted:Outdoor != e

0.88